In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [4]:
import pandas as pd

df_valid = pd.read_csv("../preprocessed/002_gpu/000/train.csv")

'While almost all astrophysicists today reject MOND in favor of dark matter, a small number of researchers continue to enhance it, recently incorporating Brans–Dicke theories into treatments that attempt to account for cosmological observations. The results are consistent with MOND, and place limits on alternative explanations involving dark matter alone. The homogeneously distributed mass of the universe would result in a roughly scalar field that permeated the universe and would serve as a source for Newton\'s gravitational constant; creating a theory of quantum gravity. === MOND === Modified Newtonian Dynamics (MOND) is a relatively modern proposal to explain the galaxy rotation problem based on a variation of Newton\'s Second Law of Dynamics at low accelerations. Topics investigated include quintessence, Modified Newtonian Dynamics (MOND) and its relativistic generalization TeVeS, and warm dark matter. ==History== Modern physical cosmology as it is currently studied first emerged a

In [12]:
text = """
In quantum mechanics, the Wigner–Weyl transform or Weyl–Wigner transform (after Hermann Weyl and Eugene Wigner) is the invertible mapping between functions in the quantum phase space formulation and Hilbert space operators in the Schrödinger picture.

Often the mapping from functions on phase space to operators is called the Weyl transform or Weyl quantization, whereas the inverse mapping, from operators to functions on phase space, is called the Wigner transform. This mapping was originally devised by Hermann Weyl in 1927 in an attempt to map symmetrized classical phase space functions to operators, a procedure known as Weyl quantization.[1] It is now understood that Weyl quantization does not satisfy all the properties one would require for consistent quantization and therefore sometimes yields unphysical answers. On the other hand, some of the nice properties described below suggest that if one seeks a single consistent procedure mapping functions on the classical phase space to operators, the Weyl quantization is the best option: a sort of normal coordinates of such maps. (Groenewold's theorem asserts that no such map can have all the ideal properties one would desire.)

Regardless, the Weyl–Wigner transform is a well-defined integral transform between the phase-space and operator representations, and yields insight into the workings of quantum mechanics. Most importantly, the Wigner quasi-probability distribution is the Wigner transform of the quantum density matrix, and, conversely, the density matrix is the Weyl transform of the Wigner function.

In contrast to Weyl's original intentions in seeking a consistent quantization scheme, this map merely amounts to a change of representation within quantum mechanics; it need not connect "classical" with "quantum" quantities. For example, the phase-space function may depend explicitly on Planck's constant ħ, as it does in some familiar cases involving angular momentum. This invertible representation change then allows one to express quantum mechanics in phase space, as was appreciated in the 1940s by Hilbrand J. Groenewold[2] and José Enrique Moyal.[3][4]

"""

encoded_input = tokenizer(text, truncation=True, max_length=384)
tokenizer.decode(encoded_input["input_ids"])

'[CLS] in quantum mechanics, the wigner – weyl transform or weyl – wigner transform ( after hermann weyl and eugene wigner ) is the invertible mapping between functions in the quantum phase space formulation and hilbert space operators in the schrodinger picture. often the mapping from functions on phase space to operators is called the weyl transform or weyl quantization, whereas the inverse mapping, from operators to functions on phase space, is called the wigner transform. this mapping was originally devised by hermann weyl in 1927 in an attempt to map symmetrized classical phase space functions to operators, a procedure known as weyl quantization. [ 1 ] it is now understood that weyl quantization does not satisfy all the properties one would require for consistent quantization and therefore sometimes yields unphysical answers. on the other hand, some of the nice properties described below suggest that if one seeks a single consistent procedure mapping functions on the classical pha

In [13]:
" ".join(text.split(" ")[:384])

'\nIn quantum mechanics, the Wigner–Weyl transform or Weyl–Wigner transform (after Hermann Weyl and Eugene Wigner) is the invertible mapping between functions in the quantum phase space formulation and Hilbert space operators in the Schrödinger picture.\n\nOften the mapping from functions on phase space to operators is called the Weyl transform or Weyl quantization, whereas the inverse mapping, from operators to functions on phase space, is called the Wigner transform. This mapping was originally devised by Hermann Weyl in 1927 in an attempt to map symmetrized classical phase space functions to operators, a procedure known as Weyl quantization.[1] It is now understood that Weyl quantization does not satisfy all the properties one would require for consistent quantization and therefore sometimes yields unphysical answers. On the other hand, some of the nice properties described below suggest that if one seeks a single consistent procedure mapping functions on the classical phase space t